<a href="https://colab.research.google.com/github/bderdz/music_mental_health/blob/main/music_mental_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analiza zbioru danych Music & Mental Health Survey

Dataset: [Music & Mental Health Survey Results](https://www.kaggle.com/datasets/catherinerasgaitis/mxmh-survey-results)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from google.colab import drive

%matplotlib inline

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/big_data/mxmh_survey_results.csv')

df.head()

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
0,8/27/2022 19:29:02,18.0,Spotify,3.0,Yes,Yes,Yes,Latin,Yes,Yes,...,Sometimes,Very frequently,Never,Sometimes,3.0,0.0,1.0,0.0,NaN,I understand.
1,8/27/2022 19:57:31,63.0,Pandora,1.5,Yes,No,No,Rock,Yes,No,...,Sometimes,Rarely,Very frequently,Rarely,7.0,2.0,2.0,1.0,NaN,I understand.
2,8/27/2022 21:28:18,18.0,Spotify,4.0,No,No,No,Video game music,No,Yes,...,Never,Rarely,Rarely,Very frequently,7.0,7.0,10.0,2.0,No effect,I understand.
3,8/27/2022 21:40:40,61.0,YouTube Music,2.5,Yes,No,Yes,Jazz,Yes,Yes,...,Sometimes,Never,Never,Never,9.0,7.0,3.0,3.0,Improve,I understand.
4,8/27/2022 21:54:47,18.0,Spotify,4.0,Yes,No,No,R&B,Yes,No,...,Very frequently,Very frequently,Never,Rarely,7.0,2.0,5.0,9.0,Improve,I understand.


# Czyszczenie zbioru danych

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Timestamp                     736 non-null    object 
 1   Age                           735 non-null    float64
 2   Primary streaming service     735 non-null    object 
 3   Hours per day                 736 non-null    float64
 4   While working                 733 non-null    object 
 5   Instrumentalist               732 non-null    object 
 6   Composer                      735 non-null    object 
 7   Fav genre                     736 non-null    object 
 8   Exploratory                   736 non-null    object 
 9   Foreign languages             732 non-null    object 
 10  BPM                           629 non-null    float64
 11  Frequency [Classical]         736 non-null    object 
 12  Frequency [Country]           736 non-null    object 
 13  Frequ

Kolumny **Timestamp** i **Permissions** nie wpływają na analizę ponieważ to zwykła formalność więc można je usunąć

In [5]:
df.drop(columns=['Timestamp', 'Permissions'], inplace=True)

In [6]:
df.dtypes

,0
Age,float64
Primary streaming service,object
Hours per day,float64
While working,object
Instrumentalist,object
Composer,object
Fav genre,object
Exploratory,object
Foreign languages,object
BPM,float64


* Takie kolumny jak **Age** i **BPM** mają typ **float64** ale powinny być raczej **int64** ( *ale najpierw musiałbym pozbyć się braków danych* )
* Jak też można zauważyć większość kolumn ma typ **String** który zamieniam na **Categorical**


---


*Aby nie powtarzać kodu wybiore indeksy kolumn mających typ* **object** *i zamienię je na kolumny z zastosowaniem na nich funkcji zmieniającej typ na* **category**

In [7]:
df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').apply(lambda col : col.astype("category"))

## Brakujące dane

In [8]:
df.isna().mean() * 100

,0
Age,0.135870
Primary streaming service,0.135870
Hours per day,0.000000
While working,0.407609
Instrumentalist,0.543478
Composer,0.135870
Fav genre,0.000000
Exploratory,0.000000
Foreign languages,0.543478
BPM,14.538043


Jak widać mamy:
* Age < 1%
* Primary streaming service	< 1%
* While working	< 1%
* Instrumentalist	< 1%
* Composer	< 1%
* Foreign languages	< 1%
* BPM ~ 14%
* Music effects ~ 1%

In [9]:
df_backup = df.copy()

In [10]:
df['Age'].isna().sum()

1

**Age** - Usuwam rekordy z brakującym wiekiem ponieważ wartości brakuje tylko w 1 rekordzie a jest ona dla nas ważna w analizie

In [11]:
df.dropna(subset=['Age'], inplace=True)

In [12]:
df.groupby('Primary streaming service')['Primary streaming service'].count()

<ipython-input-12-70b158508441>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Primary streaming service')['Primary streaming service'].count()


,Primary streaming service
Primary streaming service,
Apple Music,51
I do not use a streaming service.,71
Other streaming service,50
Pandora,11
Spotify,457
YouTube Music,94


**Primary streaming service** - wypełnię braki wartością Spotify ponieważ jest on najczęściej wybieraną platformą

In [13]:
df['Primary streaming service'] = df['Primary streaming service'].fillna('Spotify')

**While working** - wypełniam wartością **Yes** ponieważ występuje najczęściej i jest bardziej realistyczna dla wielu osób

In [14]:
df['While working'].mode()[0]

'Yes'

In [15]:
df['While working'] = df['While working'].fillna('Yes')

In [16]:
df.groupby(['Instrumentalist', 'Composer'])['Composer'].count()

<ipython-input-16-eed88122879e>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Instrumentalist', 'Composer'])['Composer'].count()


Instrumentalist  Composer
No               No          463
                 Yes          33
Yes              No          143
                 Yes          92
Name: Composer, dtype: int64

**Instrumentalist, Composer** - wypełnię wartością **No** ze względu na to że częsciej człowiek nie robi własnej muzyki i nie gra na instrumentach

In [17]:
df['Composer'] = df['Composer'].fillna('No')
df['Instrumentalist'] = df['Instrumentalist'].fillna('No')

**Foreign languages** - zostanie wypełnione metodą **Backward fill**

In [18]:
df['Foreign languages'] = df['Foreign languages'].bfill()
df['Foreign languages'].isna().sum()

0

**BPM** - ze względu na duży procent spróbuje wypełnić metodą interpolacji wielomianowej

In [19]:
df['BPM'] = df['BPM'].interpolate(method='polynomial', order=2)
df['BPM'].isna().sum()

0

**Music effects** - aby nie tracić na ilości rekordów możemy uznać że brakujące wartości to **Brak efektu (No effect)**

In [20]:
df['Music effects'] = df['Music effects'].fillna('No effect')

In [21]:
df.isna().mean() * 100

,0
Age,0.0
Primary streaming service,0.0
Hours per day,0.0
While working,0.0
Instrumentalist,0.0
Composer,0.0
Fav genre,0.0
Exploratory,0.0
Foreign languages,0.0
BPM,0.0


Teraz nie mając brakujących wartości można zamienić typ danych w kolumnach **Age** i **BPM**

In [22]:
df['Age'] = df['Age'].astype('int64')
df['BPM'] = df['BPM'].astype('int64')

## Wartości odstające
Wartości odstające możemy mieć (mamy) w kolumnie **BPM** więc użyję metody **IQR-based Outlier Detection**

In [23]:
df['BPM'].mean()

1362455.137414966

In [24]:
Q1 = df['BPM'].quantile(0.25)
Q3 = df['BPM'].quantile(0.75)

IQR = Q3 - Q1
max_value = int(Q3 + 1.5 * IQR) #kastuje na int ponieważ nie potrzebujemy zmiennoprzycinowej liczby do BPM
min_value = int(Q1 - 1.5 * IQR)

df['BPM'] = df['BPM'].apply(lambda v: min_value if v < min_value else max_value if v > max_value else v)

In [25]:
df['BPM'].mean()

123.73877551020408

Końcowy efekt:

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 735 entries, 0 to 735
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Age                           735 non-null    int64   
 1   Primary streaming service     735 non-null    category
 2   Hours per day                 735 non-null    float64 
 3   While working                 735 non-null    category
 4   Instrumentalist               735 non-null    category
 5   Composer                      735 non-null    category
 6   Fav genre                     735 non-null    category
 7   Exploratory                   735 non-null    category
 8   Foreign languages             735 non-null    category
 9   BPM                           735 non-null    int64   
 10  Frequency [Classical]         735 non-null    category
 11  Frequency [Country]           735 non-null    category
 12  Frequency [EDM]               735 non-null    category


# EDA

## Ogólne przedstawienie danych
Zaczynam EDA od zobrazowania podstawowych cech osób które wzięli udział w badaniu:
* Jak widać w większości to **osoby do 40 roku życia** i tylko nieliczna ilość przekracza ten próg. Znacząca większość nie gra na żadnym instrumencie i nie tworzy własnej muzyki.
* **Większość badanych osób słucha muzyki podczas pracy a dzienny czas słuchania wacha się najbardziej od 0 do około 5/6 godzin**

In [27]:
person_info = df[['Age','Instrumentalist','Composer','Hours per day','While working']]

figure = make_subplots(rows=2, cols=3, subplot_titles=person_info.columns)

for i, col_name in enumerate(person_info.columns):
  row = i // 3 + 1
  column = i % 3 + 1
  figure.add_trace(
    go.Histogram(x=person_info[col_name],name=col_name,nbinsx=20),
    row=row, col=column)

figure.update_layout(
    title='Histogram cech',
    showlegend=False
)

Zobaczmy jak wygląda sytuacja z chorobami psychicznymi u badanych osób:

Znacząca ilość osób ma zaburzenie lękowe, trochę lepiej ale nadal źle wygląda sytuacja z depresją. Najlepiej jest z zaburzeniem obsesyjno-kompulsyjnym.

**Co ciekawie patrząc na ilość osób cierpiących na zaburzenie lękowe albo depresje wydaje się dziwnym histogram bezsennosci, jeżeli poziom lękowy i depresji raczej zbliża się do maksymalnego to poziom bezsenności trzyma się na środku albo wcale jej nie ma co wygląda na sprzeczne ponieważ takim chorobom zazwyczaj ona towarzyszy**

In [33]:
mental_illness = df[['Anxiety','Depression','Insomnia', 'OCD']]

figure = make_subplots(rows=2, cols=2, subplot_titles=mental_illness.columns)

for i, col_name in enumerate(mental_illness.columns):
  row = i // 2 + 1
  column = i % 2 + 1
  figure.add_trace(
    go.Histogram(x=mental_illness[col_name],name=col_name,nbinsx=20),
    row=row, col=column)

figure.update_layout(
    title='Histogram chorób psychicznych',
    showlegend=False,
    width=900,
    height=500
)

Przechodząc do muzyki możemy zaobserwować że **Spotify** jest najpopularniejszą platformą wśród badanych (oczywiście nic dziwnego)

Co ciekawe więcej ludzi nie używa żadnego serwisu niż używa Apple Music

In [29]:
streaming_services = df['Primary streaming service'].value_counts().reset_index()
streaming_services.columns = ['service', 'count']

pie = px.pie(streaming_services, values='count', names='service', hole=0.2, title='Używane serwisy streamingowe', height=400, width=700)
pie.show()